In [2]:

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
csv = 'ifc_clean_tweets.csv'
my_df = pd.read_csv(csv)
my_df.head()

,Unnamed: 0,full_text,sent_score
0,0,whi wait get fine govern take care wear mask m...,-1
1,1,start monday delhi govern gave relax lockdown ...,-1
2,2,not meme just inform lockdown look like,-1
3,3,with lockdown focu reviv economi howev team vi...,-1
4,4,hello guy video get look sonam kapoor celebr i...,-1


In [4]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1748 entries, 0 to 1747
Data columns (total 3 columns):
Unnamed: 0    1748 non-null int64
full_text     1748 non-null object
sent_score    1748 non-null int64
dtypes: int64(2), object(1)
memory usage: 41.1+ KB


In [5]:

x = my_df.full_text
y = my_df.sent_score


In [6]:

from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [7]:
print ("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {2} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {2} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1713 entries with 13.25% negative, 54.17% positive
Validation set has total 35.294117647058826 entries with 5.88% negative, 35.29% positive
Test set has total 55.55555555555556 entries with 5.56% negative, 55.56% positive


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
tvec1.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=100000, min_df=1, ngram_range=(1, 3), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [9]:
x_train_tfidf = tvec1.transform(x_train)


In [10]:
x_validation_tfidf = tvec1.transform(x_validation).toarray()


In [11]:

%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train_tfidf, y_train)

Wall time: 1.54 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
clf.score(x_validation_tfidf, y_validation)


0.4117647058823529

In [13]:
clf.score(x_train_tfidf, y_train)


0.9182720373613543

In [14]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [15]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

In [16]:
%%time
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=35453))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 59ms/step - loss: 0.6147 - accuracy: 0.1524 - val_loss: 0.3024 - val_accuracy: 0.0588
Epoch 2/5
54/53 [==============================] - 3s 48ms/step - loss: 0.2138 - accuracy: 0.2154 - val_loss: -0.1777 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: -0.4283 - accuracy: 0.4081 - val_loss: -0.5778 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 51ms/step - loss: -1.2951 - accuracy: 0.5797 - val_loss: -1.2136 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 48ms/step - loss: -2.3761 - accuracy: 0.6054 - val_loss: -2.1354 - val_accuracy: 0.2353
Wall time: 14.5 s


In [17]:
 from sklearn.preprocessing import Normalizer
norm = Normalizer().fit(x_train_tfidf)
x_train_tfidf_norm = norm.transform(x_train_tfidf)
x_validation_tfidf_norm = norm.transform(x_validation_tfidf)

In [18]:
%%time
model_n = Sequential()
model_n.add(Dense(64, activation='relu', input_dim=35453))
model_n.add(Dense(1, activation='sigmoid'))
model_n.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_n.fit_generator(generator=batch_generator(x_train_tfidf_norm, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf_norm, y_validation),
                    steps_per_epoch=x_train_tfidf_norm.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 60ms/step - loss: 0.6160 - accuracy: 0.1559 - val_loss: 0.3130 - val_accuracy: 0.1176
Epoch 2/5
54/53 [==============================] - 3s 49ms/step - loss: 0.2203 - accuracy: 0.2195 - val_loss: -0.1562 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: -0.4168 - accuracy: 0.4203 - val_loss: -0.5467 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 52ms/step - loss: -1.2744 - accuracy: 0.5849 - val_loss: -1.1721 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 52ms/step - loss: -2.3373 - accuracy: 0.6095 - val_loss: -2.0762 - val_accuracy: 0.2353
Wall time: 14.7 s


In [19]:

model1 = Sequential()
model1.add(Dense(64, activation='relu', input_dim=35453))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 58ms/step - loss: 0.6235 - accuracy: 0.1500 - val_loss: 0.3362 - val_accuracy: 0.0588
Epoch 2/5
54/53 [==============================] - 3s 48ms/step - loss: 0.2855 - accuracy: 0.1874 - val_loss: -0.1106 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 50ms/step - loss: -0.2496 - accuracy: 0.3205 - val_loss: -0.4796 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 51ms/step - loss: -1.0142 - accuracy: 0.5201 - val_loss: -1.0135 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 49ms/step - loss: -1.9569 - accuracy: 0.5692 - val_loss: -1.8125 - val_accuracy: 0.2353


In [20]:
def batch_generator_shuffle(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    np.random.shuffle(index)
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            np.random.shuffle(index)
            counter=0

In [21]:

%%time
model_s = Sequential()
model_s.add(Dense(64, activation='relu', input_dim=35453))
model_s.add(Dense(1, activation='sigmoid'))
model_s.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 57ms/step - loss: 0.6208 - accuracy: 0.1635 - val_loss: 0.3312 - val_accuracy: 0.0588
Epoch 2/5
54/53 [==============================] - 3s 53ms/step - loss: 0.2567 - accuracy: 0.2055 - val_loss: -0.1059 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: -0.3377 - accuracy: 0.4069 - val_loss: -0.4910 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 49ms/step - loss: -1.1672 - accuracy: 0.5890 - val_loss: -1.0619 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 50ms/step - loss: -2.1767 - accuracy: 0.6147 - val_loss: -1.9627 - val_accuracy: 0.2353
Wall time: 14.5 s


In [22]:

%%time
model_s_1 = Sequential()
model_s_1.add(Dense(64, activation='relu', input_dim=35453))
model_s_1.add(Dropout(0.2))
model_s_1.add(Dense(1, activation='sigmoid'))
model_s_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s_1.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 58ms/step - loss: 0.6359 - accuracy: 0.1535 - val_loss: 0.3891 - val_accuracy: 0.0588
Epoch 2/5
54/53 [==============================] - 3s 51ms/step - loss: 0.3397 - accuracy: 0.1903 - val_loss: -0.0248 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: -0.1365 - accuracy: 0.3252 - val_loss: -0.3488 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 49ms/step - loss: -0.8176 - accuracy: 0.5552 - val_loss: -0.7887 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 49ms/step - loss: -1.6615 - accuracy: 0.5943 - val_loss: -1.4607 - val_accuracy: 0.2353
Wall time: 14.5 s


In [23]:

%%time
import keras
custom_adam = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_2 = Sequential()
model_testing_2.add(Dense(64, activation='relu', input_dim=35453))
model_testing_2.add(Dense(1, activation='sigmoid'))
model_testing_2.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_2.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 58ms/step - loss: 0.3987 - accuracy: 0.1570 - val_loss: -0.7945 - val_accuracy: 0.2353
Epoch 2/5
54/53 [==============================] - 3s 51ms/step - loss: -2.1261 - accuracy: 0.3847 - val_loss: -3.2655 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 54ms/step - loss: -9.8428 - accuracy: 0.5184 - val_loss: -10.6643 - val_accuracy: 0.1765
Epoch 4/5
54/53 [==============================] - 3s 50ms/step - loss: -23.4169 - accuracy: 0.5085 - val_loss: -22.3368 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 48ms/step - loss: -44.8907 - accuracy: 0.5020 - val_loss: -39.1998 - val_accuracy: 0.2353
Wall time: 14.7 s


In [24]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_3 = Sequential()
model_testing_3.add(Dense(64, activation='relu', input_dim=35453))
model_testing_3.add(Dense(1, activation='sigmoid'))
model_testing_3.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_3.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 56ms/step - loss: 0.1708 - accuracy: 0.2037 - val_loss: -1.5514 - val_accuracy: 0.2941
Epoch 2/5
54/53 [==============================] - 3s 47ms/step - loss: -8.1149 - accuracy: 0.4437 - val_loss: -11.6145 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: -37.4788 - accuracy: 0.4682 - val_loss: -42.2377 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 51ms/step - loss: -94.7732 - accuracy: 0.4565 - val_loss: -84.0767 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 49ms/step - loss: -181.8308 - accuracy: 0.4892 - val_loss: -155.4153 - val_accuracy: 0.2353
Wall time: 14.1 s


In [25]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_4 = Sequential()
model_testing_4.add(Dense(64, activation='relu', input_dim=35453))
model_testing_4.add(Dense(1, activation='sigmoid'))
model_testing_4.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_4.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 55ms/step - loss: -6.8633 - accuracy: 0.2890 - val_loss: -41.6529 - val_accuracy: 0.2941
Epoch 2/5
54/53 [==============================] - 3s 48ms/step - loss: -414.8772 - accuracy: 0.3701 - val_loss: -618.9533 - val_accuracy: 0.2941
Epoch 3/5
54/53 [==============================] - 3s 51ms/step - loss: -2348.4646 - accuracy: 0.4326 - val_loss: -2505.2078 - val_accuracy: 0.1765
Epoch 4/5
54/53 [==============================] - 3s 51ms/step - loss: -6726.8036 - accuracy: 0.4402 - val_loss: -5494.8032 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 51ms/step - loss: -14023.1971 - accuracy: 0.4752 - val_loss: -10935.0859 - val_accuracy: 0.2353
Wall time: 14.4 s


In [26]:
%%time
custom_adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model_testing_5 = Sequential()
model_testing_5.add(Dense(64, activation='relu', input_dim=35453))
model_testing_5.add(Dense(1, activation='sigmoid'))
model_testing_5.compile(optimizer=custom_adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing_5.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 3s 56ms/step - loss: 0.6614 - accuracy: 0.1956 - val_loss: 0.5450 - val_accuracy: 0.1176
Epoch 2/5
54/53 [==============================] - 3s 48ms/step - loss: 0.5158 - accuracy: 0.2055 - val_loss: 0.3626 - val_accuracy: 0.1765
Epoch 3/5
54/53 [==============================] - 3s 48ms/step - loss: 0.2834 - accuracy: 0.3287 - val_loss: 0.1796 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 3s 50ms/step - loss: -0.0340 - accuracy: 0.5575 - val_loss: 0.0061 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 3s 49ms/step - loss: -0.3958 - accuracy: 0.6270 - val_loss: -0.1634 - val_accuracy: 0.2353
Wall time: 14 s


In [27]:

%%time
model_s_2 = Sequential()
model_s_2.add(Dense(128, activation='relu', input_dim=35453))
model_s_2.add(Dense(1, activation='sigmoid'))
model_s_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_s_2.fit_generator(generator=batch_generator_shuffle(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/5
54/53 [==============================] - 5s 89ms/step - loss: 0.5844 - accuracy: 0.1705 - val_loss: 0.1418 - val_accuracy: 0.1765
Epoch 2/5
54/53 [==============================] - 5s 83ms/step - loss: 0.0229 - accuracy: 0.2353 - val_loss: -0.3740 - val_accuracy: 0.2353
Epoch 3/5
54/53 [==============================] - 4s 83ms/step - loss: -0.9851 - accuracy: 0.5231 - val_loss: -1.1358 - val_accuracy: 0.2353
Epoch 4/5
54/53 [==============================] - 4s 82ms/step - loss: -2.4874 - accuracy: 0.5750 - val_loss: -2.4438 - val_accuracy: 0.2353
Epoch 5/5
54/53 [==============================] - 5s 85ms/step - loss: -4.5941 - accuracy: 0.5750 - val_loss: -4.1981 - val_accuracy: 0.2353
Wall time: 23.4 s


Reference--- https://github.com/tthustla/twitter_sentiment_analysis_part9/blob/master/Capstone_part4-Copy7.ipynb